In [1]:
import os
import json
import zipfile
from huggingface_hub import hf_hub_download
from tqdm.auto import tqdm
from io import TextIOWrapper
import pandas as pd
from io import BytesIO

In [14]:
# Параметры
repo_id = "ajtkulov/telegram-ru"
meta_zip_filename = "meta/all.channels.csv.zip"          # путь в репозитории
output_meta_dir = "data/posts/ajtkulov/meta"             # куда сохраняем

os.makedirs(output_meta_dir, exist_ok=True)

# Шаг 1: Скачивание ZIP-файла
try:
    zip_path = hf_hub_download(
        repo_id=repo_id,
        filename=meta_zip_filename,
        repo_type="dataset"
    )
    print(f"ZIP-файл скачан: {zip_path}")
except Exception as e:
    print(f"Ошибка скачивания: {e}")
    raise

# Шаг 2: Распаковка в нужную папку
print("Распаковка в папку:", output_meta_dir)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    inner_files = zip_ref.namelist()
    print(f"Файлы внутри ZIP: {inner_files}")
    
    if not inner_files:
        raise ValueError("ZIP-файл пустой")
    
    zip_ref.extractall(output_meta_dir)
    print("Распаковка завершена")

# Шаг 3: Поиск распакованного CSV-файла
csv_path = None
for file in os.listdir(output_meta_dir):
    if file.lower().endswith('.csv'):
        csv_path = os.path.join(output_meta_dir, file)
        break

if not csv_path:
    raise FileNotFoundError("CSV-файл не найден после распаковки. Проверьте папку.")

print(f"\nЧтение CSV из файла: {csv_path}")

# Шаг 4: Чтение с обработкой всех типичных проблем
encodings = ['utf-8-sig', 'cp1251', 'utf-8', 'latin1', 'iso-8859-1']
df_channels = None

for encoding in encodings:
    try:
        df_channels = pd.read_csv(
            csv_path,
            encoding=encoding,
            sep='\t',                  # табуляция (как у вас)
            on_bad_lines='skip',       # пропуск битых строк
            low_memory=False,
            encoding_errors='replace'  # замена кракозябр
        )
        print(f"\nУСПЕХ! Файл прочитан с кодировкой: {encoding}")
        break
    except Exception as e:
        print(f"Попытка с {encoding} провалилась: {str(e)}")

if df_channels is None:
    raise ValueError("Не удалось прочитать CSV ни одной кодировкой. Возможно файл повреждён.")

# ← Добавляем названия колонок
column_names = ["link", "name", "description", "category", "message_id"]
if len(df_channels.columns) == len(column_names):
    df_channels.columns = column_names
    print("\nКолонки успешно переименованы в: link, name, description, category, message_id")
else:
    print(f"\nВнимание! Количество столбцов ({len(df_channels.columns)}) не равно ожидаемому ({len(column_names)}).")
    print("Колонки НЕ переименованы. Текущие:", df_channels.columns.tolist())


# Общая статистика
print(f"Количество каналов: {len(df_channels):,}")
print(f"Количество столбцов: {len(df_channels.columns)}")
print("Столбцы:", df_channels.columns.tolist())

# Первые 10 строк
print("\nПервые 10 каналов:")
display(df_channels.head(10))

# Распределение категорий
if 'category' in df_channels.columns:
    print("\nРаспределение по category (топ-15):")
    print(df_channels['category'].value_counts().head(150))
else:
    print("\nКолонка 'category' не найдена")

if 'category' in df_channels.columns:
    unique_categories = df_channels['category'].dropna().unique()
    unique_df = pd.DataFrame(unique_categories, columns=['category'])
    
    unique_csv_path = os.path.join(output_meta_dir, "unique_categories.csv")
    unique_df.to_csv(unique_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"\nУникальные категории сохранены в файл: {unique_csv_path}")
    print(f"Количество уникальных категорий: {len(unique_categories)}")
    print("Первые 20 уникальных категорий:")
    print(unique_df.head(20))
else:
    print("\nНе удалось сохранить категории — колонка 'category' отсутствует")

print("CSV лежит здесь:", csv_path)

ZIP-файл скачан: C:\Users\Admin\.cache\huggingface\hub\datasets--ajtkulov--telegram-ru\snapshots\e7c2668f8ffe8d7b9725d4639d3f9e96a25a58b4\meta\all.channels.csv.zip
Распаковка в папку: data/posts/ajtkulov/meta
Файлы внутри ZIP: ['all.channels.csv']
Распаковка завершена

Чтение CSV из файла: data/posts/ajtkulov/meta\all.channels.csv

УСПЕХ! Файл прочитан с кодировкой: utf-8-sig

Колонки успешно переименованы в: link, name, description, category, message_id
Количество каналов: 380,467
Количество столбцов: 5
Столбцы: ['link', 'name', 'description', 'category', 'message_id']

Первые 10 каналов:


,link,name,description,category,message_id
0,https://tgstat.ru/channel/@premium,Telegram Premium,Telegram Premium – a subscription that unlocks...,Telegram,7980100.0
1,https://tgstat.ru/channel/sOj9iDAtUkMyYWQy,Топор Live,"Нейтрально, без пропаганды. Топор Live с быстр...",Новости,4388359.0
2,https://tgstat.ru/channel/@wewantyoutodothejob,WeWantYou,Канал для поиска исполнителей для разных задач...,Другое,4158678.0
3,https://tgstat.ru/channel/@leoday,Леонардо Дайвинчик,Бот знакомств @leomatchbot,Юмор и развлечение,4057819.0
4,https://tgstat.ru/channel/@novosti_efir,Прямой Эфир • Новости,️Все самое важное в одном канале. Новости Росс...,Новости,3886208.0
5,https://tgstat.ru/channel/@novosti_voinaa,СМИ Россия не Москва,Эруктации информпространства России и ее Окраин.,Новости,3368394.0
6,https://tgstat.ru/channel/@rian_ru,РИА Новости,Главные Новости РИА t.me/rian_ru,Новости,3220976.0
7,https://tgstat.ru/channel/@invest_zonaa,INVEST ZONE,"Привет! Я Руслан, с 2017 года торгую рынок кри...",Криптовалюты,3057506.0
8,https://tgstat.ru/channel/@mash,Mash,"Прислать новость, фото, видео, аудио, бересту:...",Новости,2827677.0
9,https://tgstat.ru/channel/@crypto_drop_stukach,Дропы от Стукача,"Все о крипто раздачах, прибыльных темах и абуз...",Шок-конент,2636192.0



Распределение по category (топ-15):
category
Новости                                                               25909
Блоги                                                                 25599
Другое                                                                20693
Мода и красота                                                        17673
Психология                                                            13740
                                                                      ...  
Политика|||Регион|||Свердловская область                                 19
Политика|||Регион|||Самарская область                                    19
Путешествия|||Регион|||Приморский край                                   18
Новости|||Регион|||Кабардино-Балкарская Республика|||Новости и СМИ       18
Политика|||Регион|||Пермский край                                        18
Name: count, Length: 150, dtype: int64

Уникальные категории сохранены в файл: data/posts/ajtkulov/meta\unique_categor

In [2]:
# Ячейка: Фильтрация категорий (убираем ненужные, включая подстроки)

import pandas as pd
import os

# Путь к файлу с уникальными категориями
input_csv = 'data/posts/ajtkulov/meta/unique_categories.csv'

# Путь для сохранения отфильтрованного файла
output_csv = 'data/posts/ajtkulov/meta/filtered_categories.csv'

print("==================================================")
print(f"Чтение файла: {input_csv}")

# Читаем файл (учитывая BOM и кодировку)
df = pd.read_csv(input_csv, encoding='utf-8-sig')

print(f"Всего уникальных категорий в исходном файле: {len(df):,}")
print("Первые 10 категорий:")
print(df.head(10))

# Список запрещённых категорий и подстрок (расширенный)
FORBIDDEN_PATTERNS = [
    'Новости', 'Политика', 'Шок-контент', 'Шок-конент', 'Darknet',
    'Для взрослых', 'Эротика', 'Право', 'Религия', 'Инстаграм',
    'Другое', 'Telegram',  # слишком общие
]

# Дополнительно: любые категории, содержащие эти слова (даже с |||)
FORBIDDEN_SUBSTRINGS = ['Новости', 'Политика', 'Шок', 'Darknet']

print("\nЗапрещённые категории / подстроки (удаляем их):")
for p in FORBIDDEN_PATTERNS + FORBIDDEN_SUBSTRINGS:
    print(f" - {p}")

# Фильтрация:
# 1. Точное совпадение с запрещённым списком
# 2. Содержит любую запрещённую подстроку
mask = (
    df['category'].isin(FORBIDDEN_PATTERNS) |
    df['category'].str.contains('|'.join(FORBIDDEN_SUBSTRINGS), na=False, regex=True)
)

filtered_df = df[~mask].copy()

# Убираем дубликаты и NaN
filtered_df = filtered_df.dropna(subset=['category']).drop_duplicates(subset=['category'])

print(f"\nПосле фильтрации осталось категорий: {len(filtered_df):,}")

print("\nПервые 20 оставшихся категорий:")
print(filtered_df.head(20))

# Сохранение отфильтрованного файла
filtered_df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"\nОтфильтрованный файл сохранён: {output_csv}")
print("Теперь в нём только нишевые / полезные категории")

Чтение файла: data/posts/ajtkulov/meta/unique_categories.csv
Всего уникальных категорий в исходном файле: 1,748
Первые 10 категорий:
                                    category
0                                   Telegram
1                                    Новости
2                                     Другое
3                         Юмор и развлечение
4                               Криптовалюты
5                                 Шок-конент
6  Новости|||Регион|||Москва|||Новости и СМИ
7                                   Политика
8                                  Экономика
9                                 Технологии

Запрещённые категории / подстроки (удаляем их):
 - Новости
 - Политика
 - Шок-контент
 - Шок-конент
 - Darknet
 - Для взрослых
 - Эротика
 - Право
 - Религия
 - Инстаграм
 - Другое
 - Telegram
 - Новости
 - Политика
 - Шок
 - Darknet

После фильтрации осталось категорий: 1,417

Первые 20 оставшихся категорий:
              category
3   Юмор и развлечение
4         Крип

In [3]:
# Ячейка: Очистка первого столбца (оставляем только имя канала)

import os
import pandas as pd

# Путь к исходному файлу
input_file = 'data/posts/ajtkulov/meta/all.channels.csv'

# Путь для сохранения очищенного файла
output_file = 'data/posts/ajtkulov/meta/all_channels_clean.csv'

print("==================================================")
print(f"Чтение файла: {input_file}")

# Читаем CSV (учитываем возможные проблемы с кодировкой и разделителями)
try:
    df = pd.read_csv(
        input_file,
        encoding='utf-8-sig',
        sep='\t',                  # если табуляция
        on_bad_lines='skip',
        low_memory=False
    )
    print(f"Файл успешно прочитан. Строк: {len(df):,}")
except Exception as e:
    print(f"Ошибка чтения: {e}")
    # Попробуем другую кодировку
    df = pd.read_csv(
        input_file,
        encoding='cp1251',
        sep='\t',
        on_bad_lines='skip',
        low_memory=False
    )
    print("Успешно прочитано с cp1251")

# Проверяем, есть ли первый столбец (по умолчанию он без имени — берём по индексу)
if df.columns[0].startswith('https://tgstat.ru'):
    print("Первый столбец не имеет имени — переименовываем в 'link'")
    df = df.rename(columns={df.columns[0]: 'link'})

# Очистка первого столбца
def clean_channel_link(link):
    if pd.isna(link):
        return link
    link = str(link).strip()
    # Убираем префикс https://tgstat.ru/channel/
    if link.startswith('https://tgstat.ru/channel/'):
        link = link.replace('https://tgstat.ru/channel/', '')
    # Убираем @ в начале, если остался
    if link.startswith('@'):
        link = link[1:]
    return link

print("\nОчистка первого столбца...")
df['link'] = df['link'].apply(clean_channel_link)

# Показываем результат
print("\nПервые 10 строк после очистки:")
display(df.head(10))

# Сохранение нового файла
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\nОчищенный файл сохранён: {output_file}")
print(f"Первый столбец теперь содержит только имена каналов (без https и @)")

Чтение файла: data/posts/ajtkulov/meta/all.channels.csv
Файл успешно прочитан. Строк: 380,467
Первый столбец не имеет имени — переименовываем в 'link'

Очистка первого столбца...

Первые 10 строк после очистки:


,link,Топор 18+,Самый популярный русскоязычный Telegram канал.,Шок-конент,8179938
0,premium,Telegram Premium,Telegram Premium – a subscription that unlocks...,Telegram,7980100.0
1,sOj9iDAtUkMyYWQy,Топор Live,"Нейтрально, без пропаганды. Топор Live с быстр...",Новости,4388359.0
2,wewantyoutodothejob,WeWantYou,Канал для поиска исполнителей для разных задач...,Другое,4158678.0
3,leoday,Леонардо Дайвинчик,Бот знакомств @leomatchbot,Юмор и развлечение,4057819.0
4,novosti_efir,Прямой Эфир • Новости,️Все самое важное в одном канале. Новости Росс...,Новости,3886208.0
5,novosti_voinaa,СМИ Россия не Москва,Эруктации информпространства России и ее Окраин.,Новости,3368394.0
6,rian_ru,РИА Новости,Главные Новости РИА t.me/rian_ru,Новости,3220976.0
7,invest_zonaa,INVEST ZONE,"Привет! Я Руслан, с 2017 года торгую рынок кри...",Криптовалюты,3057506.0
8,mash,Mash,"Прислать новость, фото, видео, аудио, бересту:...",Новости,2827677.0
9,crypto_drop_stukach,Дропы от Стукача,"Все о крипто раздачах, прибыльных темах и абуз...",Шок-конент,2636192.0



Очищенный файл сохранён: data/posts/ajtkulov/meta/all_channels_clean.csv
Первый столбец теперь содержит только имена каналов (без https и @)


In [ ]:


# Параметры (настройте под себя)
repo_id = "ajtkulov/telegram-ru"
output_dir = "data/posts/ajtkulov"           # Папка для результатов (CSV/JSONL)
cache_dir = os.path.join(output_dir, "cache")  # Кэш для ZIP (можно удалять после обработки)
progress_file = os.path.join(output_dir, "progress.json")  # Для паузы/продолжения

os.makedirs(output_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "csv"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "jsonl"), exist_ok=True)

# Критерии фильтра (пример; настройте)
min_text_length = 100
keywords = []  # Если пусто — без фильтра по словам; иначе содержит хотя бы одно
min_date = "2020-01-01"  # Формат YYYY-MM-DD

# Список всех файлов
file_names = [f"tg.{i}.zip" for i in range(359)]  # 0 to 112

# Для теста: file_names = file_names[:3]  # Только первые 3

# Загрузка прогресса (какие ZIP уже обработаны)
processed_files = set()
if os.path.exists(progress_file):
    with open(progress_file, 'r') as pf:
        processed_files = set(json.load(pf))
    print(f"Загружен прогресс: {len(processed_files)} файлов уже обработано")

# Основной цикл: последовательная обработка
for filename in tqdm(file_names, desc="Обработка ZIP-файлов"):
    if filename in processed_files:
        print(f"\n{filename} уже обработан — пропуск")
        continue
    
    print(f"\n=== Обработка {filename} ===")
    
    try:
        # Шаг 1: Скачивание (потоково, но hf_hub_download скачивает целиком)
        file_path = hf_hub_download(
            repo_id=repo_id,
            filename=f"data/{filename}",
            repo_type="dataset",
            cache_dir=cache_dir
        )
        print(f"Скачано в: {file_path}")
        
        # Шаг 2: Потоковая обработка ZIP
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            inner_files = [f for f in zip_ref.namelist() if not f.endswith('/') and not f.endswith('.done')]
            print(f"Файлов внутри: {len(inner_files)}")
            
            filtered_posts = []  # Временный буфер для этого ZIP (маленький, чтобы не жрать RAM)
            
            for inner_path in tqdm(inner_files, desc="Файлы внутри ZIP", leave=False):
                with zip_ref.open(inner_path) as f:
                    # Потоковое чтение строк
                    for line in TextIOWrapper(f, encoding='utf-8', errors='ignore'):
                        line = line.strip()
                        if not line: continue
                        
                        try:
                            obj = json.loads(line)
                            text = obj.get('text', '')
                            date = obj.get('date', '')
                            
                            # Фильтр
                            if (len(text) >= min_text_length and
                                (not keywords or any(kw.lower() in text.lower() for kw in keywords)) and
                                date >= min_date):
                                filtered_posts.append(obj)
                        except json.JSONDecodeError:
                            continue  # Пропуск битых строк
            
            # Шаг 3: Сохранение отфильтрованных из этого ZIP
            if filtered_posts:
                df = pd.DataFrame(filtered_posts)
                batch_name = filename.replace('.zip', '')
                
                # CSV
                csv_path = os.path.join(output_dir, "csv", f"{batch_name}.csv")
                df.to_csv(csv_path, index=False, encoding='utf-8-sig')
                print(f"CSV: {csv_path} ({len(df):,} постов)")
                
                # JSONL
                jsonl_path = os.path.join(output_dir, "jsonl", f"{batch_name}.jsonl")
                with open(jsonl_path, 'w', encoding='utf-8') as jf:
                    for post in filtered_posts:
                        jf.write(json.dumps(post, ensure_ascii=False) + '\n')
                print(f"JSONL: {jsonl_path} ({len(filtered_posts):,} постов)")
            else:
                print("Нет подходящих постов в этом ZIP")
        
        # Опционально: удалить ZIP после обработки
        os.remove(file_path)
        print(f"ZIP удалён: {file_path}")
        
        # Сохранить прогресс
        processed_files.add(filename)
        with open(progress_file, 'w') as pf:
            json.dump(list(processed_files), pf)
        print("Прогресс сохранён")
        
    except Exception as e:
        print(f"Ошибка с {filename}: {e}")
        # Здесь можно добавить паузу или break, но код продолжит с следующим

print("\nОбработка завершена!")

Загружен прогресс: 2 файлов уже обработано


Обработка ZIP-файлов:   0%|          | 0/359 [00:00<?, ?it/s]


tg.0.zip уже обработан — пропуск

tg.1.zip уже обработан — пропуск

=== Обработка tg.2.zip ===


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tg.2.zip:  18%|#8        | 241M/1.31G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# ЯЧЕЙКА: Стратифицированный отбор постов (5 на канал, топ по просмотрам, длина 150–400)

import os
import json
import zipfile
from huggingface_hub import hf_hub_download
from tqdm.auto import tqdm
from io import TextIOWrapper
import pandas as pd
import random

# ==============================================
# НАСТРОЙКИ (изменяйте здесь)
# ==============================================

repo_id = "ajtkulov/telegram-ru"

output_dir = "data/posts/ajtkulov/selected"               # куда сохранять финальный результат
cache_dir = os.path.join(output_dir, "cache")             # временный кэш для ZIP
progress_file = os.path.join(output_dir, "progress.json") # какие ZIP уже обработаны
stats_file = os.path.join(output_dir, "channel_top_posts.json")  # топ-посты по каналам

os.makedirs(output_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)

# Запрещённые категории (не берём посты из этих каналов)
forbidden_categories = [
    'Новости', 'Новости и СМИ', 'Политика', 'Шок-контент',
    # Добавьте сюда всё, что считаете новостным/мусорным
]

# Желаемое количество постов на канал
MAX_POSTS_PER_CHANNEL = 5

# Фильтр по длине текста
MIN_TEXT_LEN = 150
MAX_TEXT_LEN = 400

# Глобальный лимит записей (не превысим)
MAX_TOTAL_RECORDS = 700000

# Список ZIP-файлов (0–112, но можно ограничить)
file_names = [f"tg.{i}.zip" for i in range(230)]
# file_names = file_names[:5]  # ← для теста

# ==============================================
# Вспомогательные структуры
# ==============================================

# channel → list of (views, text, obj) — храним только топ-5 по просмотрам
channel_top_posts = {}

# Загрузка существующей статистики, если есть
if os.path.exists(stats_file):
    with open(stats_file, 'r', encoding='utf-8') as f:
        channel_top_posts = json.load(f)
    print(f"Загружено статистики по {len(channel_top_posts):,} каналам")

# Прогресс: какие ZIP уже обработаны
processed_files = set()
if os.path.exists(progress_file):
    with open(progress_file, 'r') as pf:
        processed_files = set(json.load(pf))
    print(f"Уже обработано ZIP-файлов: {len(processed_files)}")

# ==============================================
# Основной цикл: чтение и отбор
# ==============================================

total_selected = 0

for filename in tqdm(file_names, desc="Обработка ZIP-файлов"):
    if filename in processed_files:
        print(f"{filename} уже обработан — пропуск")
        continue
    
    print(f"\n=== {filename} ===")
    
    try:
        file_path = hf_hub_download(
            repo_id=repo_id,
            filename=f"data/{filename}",
            repo_type="dataset",
            cache_dir=cache_dir
        )
        print(f"Скачано: {file_path}")
        
        with zipfile.ZipFile(file_path, 'r') as z:
            inner_files = [f for f in z.namelist() if not f.endswith('/') and not f.endswith('.done')]
            print(f"Внутри ZIP файлов: {len(inner_files)}")
            
            for inner_path in tqdm(inner_files, desc="Внутренние файлы", leave=False):
                with z.open(inner_path) as f:
                    for line in TextIOWrapper(f, encoding='utf-8', errors='ignore'):
                        line = line.strip()
                        if not line: continue
                        
                        try:
                            obj = json.loads(line)
                            channel = obj.get('channel')
                            text = obj.get('text', '')
                            views = obj.get('views', 0)  # или 'view_count', 'views_count' — проверьте
                            if not isinstance(views, (int, float)):
                                views = 0
                            
                            if not channel or not text:
                                continue
                            
                            # Фильтр по длине
                            if not (MIN_TEXT_LEN <= len(text) <= MAX_TEXT_LEN):
                                continue
                            
                            # Если канал уже в запрещённых — пропускаем
                            # (здесь нужен df_channels из мета-CSV — подключаем ниже)
                            
                            # Сохраняем пост для канала
                            if channel not in channel_top_posts:
                                channel_top_posts[channel] = []
                            
                            channel_top_posts[channel].append((views, text, obj))
                            
                            total_selected += 1
                            if total_selected >= MAX_TOTAL_RECORDS * 1.5:  # небольшой запас
                                print("Достигнут глобальный лимит — прерываем")
                                raise StopIteration
                                
                        except json.JSONDecodeError:
                            continue
        
        # После обработки ZIP — сохраняем прогресс
        processed_files.add(filename)
        with open(progress_file, 'w', encoding='utf-8') as pf:
            json.dump(list(processed_files), pf, indent=2)  # добавил indent для читаемости
        
        # Сохраняем промежуточную статистику топ-постов
        with open(stats_file, 'w', encoding='utf-8') as sf:
            # Форматируем JSON для читаемости
            formatted_stats = {}
            for channel, posts in channel_top_posts.items():
                formatted_stats[channel] = posts
            
            json.dump(formatted_stats, sf, ensure_ascii=False, indent=2)  # добавил indent
        
        # Удаляем ZIP, чтобы не занимать место
        os.remove(file_path)
        print(f"ZIP удалён: {file_path}")
        
    except StopIteration:
        break
    except Exception as e:
        print(f"Ошибка {filename}: {e}")

# ==============================================
# Финальный отбор: 5 постов на канал, топ по просмотрам
# ==============================================

print("\nФинальный отбор постов...")

selected_posts = []

for channel, post_list in tqdm(channel_top_posts.items(), desc="Отбор по каналам"):
    # Сортируем по просмотрам descending
    post_list.sort(key=lambda x: x[0], reverse=True)
    
    # Берём не более 5
    take = min(5, len(post_list))
    for _, _, obj in post_list[:take]:
        selected_posts.append(obj)
        
        if len(selected_posts) >= MAX_TOTAL_RECORDS:
            break
    
    if len(selected_posts) >= MAX_TOTAL_RECORDS:
        break

print(f"\nОтобрано постов: {len(selected_posts):,}")

# ==============================================
# Сохранение финального результата
# ==============================================

if selected_posts:
    df_final = pd.DataFrame(selected_posts)
    
    # Сохраняем в CSV
    csv_path = os.path.join(output_dir, "selected_posts.csv")
    df_final.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"Финальный CSV: {csv_path} ({len(df_final):,} строк)")
    
    # Сохраняем в JSONL (одна строка = один пост)
    jsonl_path = os.path.join(output_dir, "selected_posts.jsonl")
    with open(jsonl_path, 'w', encoding='utf-8') as jf:
        for post in selected_posts:
            jf.write(json.dumps(post, ensure_ascii=False) + '\n')
    print(f"Финальный JSONL: {jsonl_path} ({len(selected_posts):,} записей)")
    
    # Также сохраняем в красивый JSON файл для удобства чтения
    json_path = os.path.join(output_dir, "selected_posts_pretty.json")
    with open(json_path, 'w', encoding='utf-8') as jf:
        json.dump(selected_posts, jf, ensure_ascii=False, indent=2)
    print(f"Форматированный JSON: {json_path}")
    
else:
    print("Не удалось отобрать ни одного поста")

print("\nГотово!")

Обработка ZIP-файлов:   0%|          | 0/113 [00:00<?, ?it/s]


=== tg.0.zip ===


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tg.0.zip:  30%|###       | 682M/2.26G [00:00<?, ?B/s]

Скачано: data/posts/ajtkulov/selected\cache\datasets--ajtkulov--telegram-ru\snapshots\e7c2668f8ffe8d7b9725d4639d3f9e96a25a58b4\data\tg.0.zip
Внутри ZIP файлов: 891


Внутренние файлы:   0%|          | 0/891 [00:00<?, ?it/s]

Достигнут глобальный лимит — прерываем

Финальный отбор постов...


Отбор по каналам:   0%|          | 0/295 [00:00<?, ?it/s]


Отобрано постов: 1,450
Финальный CSV: data/posts/ajtkulov/selected\selected_posts.csv (1,450 строк)
Финальный JSONL: data/posts/ajtkulov/selected\selected_posts.jsonl (1,450 записей)
Форматированный JSON: data/posts/ajtkulov/selected\selected_posts_pretty.json

Готово!


In [8]:
import os
import json
import zipfile
from huggingface_hub import hf_hub_download
from tqdm.auto import tqdm
from io import TextIOWrapper
import pandas as pd
import random

# ==============================================
# НАСТРОЙКИ (изменяйте здесь)
# ==============================================

repo_id = "ajtkulov/telegram-ru"

output_dir = "data/posts/ajtkulov/selected"               # куда сохранять финальный результат
cache_dir = os.path.join(output_dir, "cache")             # временный кэш для ZIP
progress_file = os.path.join(output_dir, "progress.json") # какие ZIP уже обработаны
stats_file = os.path.join(output_dir, "channel_top_posts.json")  # топ-посты по каналам
selected_posts_file = os.path.join(output_dir, "selected_posts_temp.jsonl")

os.makedirs(output_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)

# Пути к файлам с категориями
channels_file = "data/posts/ajtkulov/meta/all_channels_clean.csv"
categories_file = "data/posts/ajtkulov/meta/filtered_categories.csv"

# Проверка существования файлов
print("Загрузка информации о каналах и категориях...")
if not os.path.exists(channels_file):
    print(f"ОШИБКА: Файл не найден: {channels_file}")
    print("Убедитесь, что файл all_channels_clean.csv находится в папке data/posts/ajtkulov/")
    exit()

if not os.path.exists(categories_file):
    print(f"ОШИБКА: Файл не найден: {categories_file}")
    print("Убедитесь, что файл filtered_categories.csv находится в папке data/posts/ajtkulov/")
    exit()

# Загружаем данные о каналах и категориях
all_channels_clean = pd.read_csv(channels_file)
filtered_categories = pd.read_csv(categories_file)

print(f"Загружено каналов: {len(all_channels_clean)}")
print(f"Загружено категорий для фильтрации: {len(filtered_categories)}")

# Создаем словарь channel -> category
channel_to_category = {}
if 'category' in all_channels_clean.columns and 'channel' in all_channels_clean.columns:
    for _, row in all_channels_clean.iterrows():
        channel_to_category[row['channel']] = row['category']
    print(f"Создан словарь категорий для {len(channel_to_category)} каналов")
else:
    print("ОШИБКА: В файле all_channels_clean.csv должны быть колонки 'channel' и 'category'")
    print(f"Найдены колонки: {list(all_channels_clean.columns)}")
    exit()

# Создаем множество разрешенных категорий
if 'category' in filtered_categories.columns:
    allowed_categories = set(filtered_categories['category'].dropna().astype(str).tolist())
    print(f"Разрешено {len(allowed_categories)} категорий")
    print("Первые 10 разрешенных категорий:")
    for i, cat in enumerate(list(allowed_categories)[:10]):
        print(f"  {i+1}. {cat}")
else:
    print("ОШИБКА: В файле filtered_categories.csv должна быть колонка 'category'")
    print(f"Найдены колонки: {list(filtered_categories.columns)}")
    exit()

Загрузка информации о каналах и категориях...
Загружено каналов: 380468
Загружено категорий для фильтрации: 1417
Создан словарь категорий для 380468 каналов
Разрешено 1417 категорий
Первые 10 разрешенных категорий:
  1. Регион|||Амурская область|||Технологии
  2. Telegram|||Регион|||Приморский край
  3. Образование|||Регион|||Республика Башкортостан
  4. Медицина|||Регион|||Ивановская область
  5. Здоровье и фитнес|||Регион|||Томская область|||Здоровье и Фитнес
  6. Блоги|||Регион|||Ямало-Ненецкий автономный округ
  7. Спорт
  8. Другое|||Регион|||Белгородская область
  9. Регион|||СтавропольѸй край|||Экономика
  10. Регион|||Республика Татарстан|||Семья и дети


In [9]:
# Желаемое количество постов на канал
MAX_POSTS_PER_CHANNEL = 5

# Фильтр по длине текста
MIN_TEXT_LEN = 150
MAX_TEXT_LEN = 400

# Глобальный лимит записей (не превысим)
MAX_TOTAL_RECORDS = 700000

# Список ZIP-файлов (0–112, но можно ограничить)
file_names = [f"tg.{i}.zip" for i in range(230)]
# file_names = file_names[:5]  # ← для теста

# ==============================================
# Вспомогательные структуры
# ==============================================

# channel → list of (views, text, obj) — храним только топ-5 по просмотрам
channel_top_posts = {}

# Кэш проверенных каналов (чтобы не проверять категорию каждый раз)
verified_channels_cache = {}

# Функция проверки, проходит ли канал по категории
def is_channel_allowed(channel_name):
    if channel_name in verified_channels_cache:
        return verified_channels_cache[channel_name]
    
    # Получаем категорию канала
    category = channel_to_category.get(channel_name)
    
    # Если категория не найдена или пустая — пропускаем
    if not category or pd.isna(category):
        verified_channels_cache[channel_name] = False
        return False
    
    # Проверяем, находится ли категория в разрешенных
    is_allowed = str(category) in allowed_categories
    verified_channels_cache[channel_name] = is_allowed
    
    return is_allowed

# Загрузка существующей статистики, если есть
if os.path.exists(stats_file):
    with open(stats_file, 'r', encoding='utf-8') as f:
        channel_top_posts = json.load(f)
    print(f"Загружено статистики по {len(channel_top_posts):,} каналам")

# Загрузка кэша проверенных каналов
cache_file = os.path.join(output_dir, "verified_channels_cache.json")
if os.path.exists(cache_file):
    with open(cache_file, 'r', encoding='utf-8') as f:
        verified_channels_cache = json.load(f)
    print(f"Загружен кэш для {len(verified_channels_cache)} каналов")

# Загрузка уже отобранных постов, если есть
selected_posts_count = 0
if os.path.exists(selected_posts_file):
    # Просто посчитаем количество строк
    with open(selected_posts_file, 'r', encoding='utf-8') as f:
        selected_posts_count = sum(1 for _ in f)
    print(f"Уже отобрано постов из предыдущих запусков: {selected_posts_count:,}")

# Прогресс: какие ZIP уже обработаны
processed_files = set()
if os.path.exists(progress_file):
    with open(progress_file, 'r') as pf:
        processed_files = set(json.load(pf))
    print(f"Уже обработано ZIP-файлов: {len(processed_files)}")

# ==============================================
# Основной цикл: чтение и отбор
# ==============================================

total_processed = 0
allowed_channels_count = 0

for filename in tqdm(file_names, desc="Обработка ZIP-файлов"):
    if filename in processed_files:
        print(f"{filename} уже обработан — пропуск")
        continue
    
    print(f"\n=== {filename} ===")
    
    try:
        file_path = hf_hub_download(
            repo_id=repo_id,
            filename=f"data/{filename}",
            repo_type="dataset",
            cache_dir=cache_dir
        )
        print(f"Скачано: {file_path}")
        
        # Открываем файл для добавления отобранных постов (режим 'a' - append)
        with open(selected_posts_file, 'a', encoding='utf-8') as spf:
            with zipfile.ZipFile(file_path, 'r') as z:
                inner_files = [f for f in z.namelist() if not f.endswith('/') and not f.endswith('.done')]
                print(f"Внутри ZIP файлов: {len(inner_files)}")
                
                for inner_path in tqdm(inner_files, desc="Внутренние файлы", leave=False):
                    with z.open(inner_path) as f:
                        for line in TextIOWrapper(f, encoding='utf-8', errors='ignore'):
                            line = line.strip()
                            if not line: continue
                            
                            total_processed += 1
                            
                            try:
                                obj = json.loads(line)
                                channel = obj.get('channel')
                                text = obj.get('text', '')
                                views = obj.get('views', 0)
                                
                                # Конвертируем views в число
                                if isinstance(views, str):
                                    if 'K' in views:
                                        views = float(views.replace('K', '').replace(',', '.')) * 1000
                                    elif 'M' in views:
                                        views = float(views.replace('M', '').replace(',', '.')) * 1000000
                                    else:
                                        try:
                                            views = float(views.replace(',', ''))
                                        except:
                                            views = 0
                                
                                if not isinstance(views, (int, float)):
                                    views = 0
                                
                                if not channel or not text:
                                    continue
                                
                                # Фильтр по длине
                                text_len = len(text)
                                if not (MIN_TEXT_LEN <= text_len <= MAX_TEXT_LEN):
                                    continue
                                
                                # Фильтр по категории канала
                                if not is_channel_allowed(channel):
                                    continue
                                
                                # Сохраняем пост для канала
                                if channel not in channel_top_posts:
                                    channel_top_posts[channel] = []
                                    allowed_channels_count += 1
                                
                                channel_top_posts[channel].append((views, text, obj))
                                
                                # Если у канала слишком много постов, оставляем только топ-N
                                if len(channel_top_posts[channel]) > MAX_POSTS_PER_CHANNEL * 2:
                                    channel_top_posts[channel].sort(key=lambda x: x[0], reverse=True)
                                    channel_top_posts[channel] = channel_top_posts[channel][:MAX_POSTS_PER_CHANNEL * 2]
                                
                                # Периодический вывод статистики
                                if total_processed % 100000 == 0:
                                    # Подсчитываем текущее количество отобранных постов
                                    current_selected = sum(min(MAX_POSTS_PER_CHANNEL, len(posts)) for posts in channel_top_posts.values())
                                    print(f"Обработано: {total_processed:,} | Каналов с разрешенной категорией: {allowed_channels_count:,} | Потенциальных отобранных: {current_selected:,}")
                                
                                if len(channel_top_posts) > 0:
                                    # Подсчитываем примерное количество отобранных постов
                                    estimated_selected = sum(min(MAX_POSTS_PER_CHANNEL, len(posts)) for posts in channel_top_posts.values())
                                    if estimated_selected >= MAX_TOTAL_RECORDS:
                                        print(f"Достигнут глобальный лимит ({estimated_selected:,} записей) — прерываем")
                                        raise StopIteration
                                        
                            except json.JSONDecodeError:
                                continue
                            except Exception as e:
                                continue
        
        # После обработки каждого ZIP — проводим промежуточный отбор
        print("Промежуточный отбор лучших постов...")
        
        # Сортируем каналы по суммарным просмотрам
        channel_stats = []
        for channel, post_list in channel_top_posts.items():
            if post_list:
                total_views = sum(views for views, _, _ in post_list[:MAX_POSTS_PER_CHANNEL])
                channel_stats.append((channel, total_views, len(post_list)))
        
        # Сортируем каналы по популярности
        channel_stats.sort(key=lambda x: x[1], reverse=True)
        
        # Отбираем посты и сохраняем их
        intermediate_selected = []
        for channel, total_views, post_count in channel_stats:
            if channel not in channel_top_posts:
                continue
            
            post_list = channel_top_posts[channel]
            post_list.sort(key=lambda x: x[0], reverse=True)
            
            take = min(MAX_POSTS_PER_CHANNEL, len(post_list))
            for _, _, obj in post_list[:take]:
                intermediate_selected.append(obj)
                
                if len(intermediate_selected) >= MAX_TOTAL_RECORDS:
                    break
            
            if len(intermediate_selected) >= MAX_TOTAL_RECORDS:
                break
        
        # Сохраняем отобранные посты в файл
        with open(selected_posts_file, 'a', encoding='utf-8') as spf:
            for post in intermediate_selected:
                spf.write(json.dumps(post, ensure_ascii=False) + '\n')
        
        # Обновляем счетчик отобранных постов
        selected_posts_count += len(intermediate_selected)
        
        # Сохраняем прогресс
        processed_files.add(filename)
        with open(progress_file, 'w', encoding='utf-8') as pf:
            json.dump(list(processed_files), pf, indent=2)
        
        # Сохраняем промежуточную статистику топ-постов
        with open(stats_file, 'w', encoding='utf-8') as sf:
            json.dump(channel_top_posts, sf, ensure_ascii=False, indent=2)
        
        # Сохраняем кэш проверенных каналов
        with open(cache_file, 'w', encoding='utf-8') as cf:
            json.dump(verified_channels_cache, cf, ensure_ascii=False, indent=2)
        
        # Удаляем ZIP, чтобы не занимать место
        os.remove(file_path)
        print(f"ZIP удалён: {file_path}")
        
        print(f"Статистика после обработки {filename}:")
        print(f"  - Обработано записей: {total_processed:,}")
        print(f"  - Отобрано постов всего: {selected_posts_count:,}")
        print(f"  - Каналов с разрешенной категорией: {allowed_channels_count:,}")
        
        # Проверяем лимит
        if selected_posts_count >= MAX_TOTAL_RECORDS:
            print(f"Достигнут глобальный лимит в {MAX_TOTAL_RECORDS:,} записей — завершаем обработку")
            break
        
    except StopIteration:
        print("Обработка прервана по достижению лимита")
        break
    except Exception as e:
        print(f"Ошибка {filename}: {e}")
        import traceback
        traceback.print_exc()

# ==============================================
# Финальный отбор и сохранение результатов
# ==============================================

print("\nФинальный отбор постов...")
print(f"Всего каналов с разрешенной категорией: {len(channel_top_posts):,}")
print(f"Уже отобрано постов: {selected_posts_count:,}")

# Загружаем все отобранные посты из временного файла
selected_posts = []
if os.path.exists(selected_posts_file):
    with open(selected_posts_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    selected_posts.append(json.loads(line))
                except:
                    continue

# Если нужно ограничить по лимиту (на случай если превысили)
if len(selected_posts) > MAX_TOTAL_RECORDS:
    print(f"Ограничиваем количество постов с {len(selected_posts):,} до {MAX_TOTAL_RECORDS:,}")
    selected_posts = selected_posts[:MAX_TOTAL_RECORDS]

print(f"\nФинальное количество постов: {len(selected_posts):,}")
print(f"Всего обработано записей: {total_processed:,}")

# ==============================================
# Сохранение финального результата
# ==============================================

if selected_posts:
    df_final = pd.DataFrame(selected_posts)
    
    # Сохраняем в CSV
    csv_path = os.path.join(output_dir, "selected_posts.csv")
    df_final.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"Финальный CSV: {csv_path} ({len(df_final):,} строк)")
    
    # Сохраняем в JSONL (одна строка = один пост)
    jsonl_path = os.path.join(output_dir, "selected_posts.jsonl")
    with open(jsonl_path, 'w', encoding='utf-8') as jf:
        for post in selected_posts:
            jf.write(json.dumps(post, ensure_ascii=False) + '\n')
    print(f"Финальный JSONL: {jsonl_path} ({len(selected_posts):,} записей)")
    
    # Сохраняем в ДВА JSON формата
    
    # 1. Красивый JSON с отступами (для чтения)
    json_path_pretty = os.path.join(output_dir, "selected_posts_pretty.json")
    with open(json_path_pretty, 'w', encoding='utf-8') as jf:
        json.dump(selected_posts, jf, ensure_ascii=False, indent=2)
    print(f"Форматированный JSON: {json_path_pretty}")
    
    # 2. Обычный JSON без отступов (одна строка, компактный)
    json_path_compact = os.path.join(output_dir, "selected_posts.json")
    with open(json_path_compact, 'w', encoding='utf-8') as jf:
        json.dump(selected_posts, jf, ensure_ascii=False)
    print(f"Компактный JSON: {json_path_compact} ({len(selected_posts):,} записей)")
    
    # Сохраняем статистику обработки
    stats_summary = {
        "total_selected_posts": len(selected_posts),
        "total_processed_records": total_processed,
        "total_allowed_channels": len(channel_top_posts),
        "max_posts_per_channel": MAX_POSTS_PER_CHANNEL,
        "min_text_length": MIN_TEXT_LEN,
        "max_text_length": MAX_TEXT_LEN,
        "max_total_records_limit": MAX_TOTAL_RECORDS,
        "allowed_categories_count": len(allowed_categories),
        "processed_zip_files": len(processed_files),
        "estimated_memory_saved_MB": (total_processed - len(selected_posts)) * 0.5 / 1024  # примерная оценка
    }
    
    summary_path = os.path.join(output_dir, "processing_summary.json")
    with open(summary_path, 'w', encoding='utf-8') as sf:
        json.dump(stats_summary, sf, ensure_ascii=False, indent=2)
    print(f"Статистика обработки: {summary_path}")
    
    # Удаляем временный файл
    if os.path.exists(selected_posts_file):
        os.remove(selected_posts_file)
        print(f"Временный файл удалён: {selected_posts_file}")
    
else:
    print("Не удалось отобрать ни одного поста")

print("\nГотово!")

Загружено статистики по 630 каналам
Загружен кэш для 868 каналов
Уже обработано ZIP-файлов: 1


Обработка ZIP-файлов:   0%|          | 0/230 [00:00<?, ?it/s]

tg.0.zip уже обработан — пропуск

=== tg.1.zip ===
Скачано: data/posts/ajtkulov/selected\cache\datasets--ajtkulov--telegram-ru\snapshots\e7c2668f8ffe8d7b9725d4639d3f9e96a25a58b4\data\tg.1.zip
Внутри ZIP файлов: 887


Внутренние файлы:   0%|          | 0/887 [00:00<?, ?it/s]

Обработано: 400,000 | Каналов с разрешенной категорией: 38 | Потенциальных отобранных: 3,287
Обработано: 1,800,000 | Каналов с разрешенной категорией: 197 | Потенциальных отобранных: 4,072
Обработано: 2,400,000 | Каналов с разрешенной категорией: 235 | Потенциальных отобранных: 4,258
Обработано: 2,900,000 | Каналов с разрешенной категорией: 304 | Потенциальных отобранных: 4,595
Обработано: 3,000,000 | Каналов с разрешенной категорией: 318 | Потенциальных отобранных: 4,664
Обработано: 6,100,000 | Каналов с разрешенной категорией: 584 | Потенциальных отобранных: 5,973
Обработано: 6,400,000 | Каналов с разрешенной категорией: 635 | Потенциальных отобранных: 6,228
Обработано: 6,700,000 | Каналов с разрешенной категорией: 647 | Потенциальных отобранных: 6,286
Промежуточный отбор лучших постов...
ZIP удалён: data/posts/ajtkulov/selected\cache\datasets--ajtkulov--telegram-ru\snapshots\e7c2668f8ffe8d7b9725d4639d3f9e96a25a58b4\data\tg.1.zip
Статистика после обработки tg.1.zip:
  - Обработано за

...

Внутренние файлы:   0%|          | 0/907 [00:00<?, ?it/s]

Обработано: 13,800,000 | Каналов с разрешенной категорией: 1,435 | Потенциальных отобранных: 10,127
Обработано: 15,200,000 | Каналов с разрешенной категорией: 1,571 | Потенциальных отобранных: 10,791
Обработано: 16,200,000 | Каналов с разрешенной категорией: 1,692 | Потенциальных отобранных: 11,375
Обработано: 16,600,000 | Каналов с разрешенной категорией: 1,715 | Потенциальных отобранных: 11,488
Обработано: 17,900,000 | Каналов с разрешенной категорией: 1,852 | Потенциальных отобранных: 12,167
Обработано: 19,000,000 | Каналов с разрешенной категорией: 1,962 | Потенциальных отобранных: 12,700
Промежуточный отбор лучших постов...
ZIP удалён: data/posts/ajtkulov/selected\cache\datasets--ajtkulov--telegram-ru\snapshots\e7c2668f8ffe8d7b9725d4639d3f9e96a25a58b4\data\tg.3.zip
Статистика после обработки tg.3.zip:
  - Обработано записей: 19,359,815
  - Отобрано постов всего: 28,955
  - Каналов с разрешенной категорией: 2,000

=== tg.4.zip ===


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tg.4.zip:   0%|          | 0.00/1.64G [00:00<?, ?B/s]

KeyboardInterrupt: 